<a href="https://colab.research.google.com/github/d9ria/SQL-G-Practice/blob/main/SQL_50_LeetCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
def to_sql(data, table_name):
  data.to_sql(table_name, con, index=False, if_exists='replace')

####1. Recyclable and Low Fat Products
Write a solution to find the ids of products that are both low fat and recyclable.

In [ ]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})
products

,product_id,low_fats,recyclable
0,0,Y,N
1,1,Y,Y
2,2,N,Y
3,3,Y,Y
4,4,N,N


In [ ]:
to_sql(products, 'products')

In [ ]:
sql = '''select t.product_id from products t
where (t.low_fats is "Y" and t.recyclable is "Y")'''
select(sql)

,product_id
0,1
1,3


####2. Find Customer Referee
Find the names of the customer that are not referred by the customer with id = 2.

In [ ]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype({'id':'Int64', 'name':'object', 'referee_id':'Int64'})
customer

,id,name,referee_id
0,1,Will,<NA>
1,2,Jane,<NA>
2,3,Alex,2
3,4,Bill,<NA>
4,5,Zack,1
5,6,Mark,2


In [ ]:
to_sql(customer, 'customer')

In [ ]:
sql = '''select t.name from customer t
where coalesce(t.referee_id, 0) != 2'''
select(sql)

,name
0,Will
1,Jane
2,Bill
3,Zack


####3. Big Countries
A country is big if:
- it has an area of at least three million (i.e., 3000000 km2), or
- it has a population of at least twenty-five million (i.e., 25000000).

Write a solution to find the name, population, and area of the big countries.

In [ ]:
data = [['Afghanistan', 'Asia', 652230, 25500100, 20343000000],
        ['Albania', 'Europe', 28748, 2831741, 12960000000],
        ['Algeria', 'Africa', 2381741, 37100000, 188681000000],
        ['Andorra', 'Europe', 468, 78115, 3712000000],
        ['Angola', 'Africa', 1246700, 20609294, 100990000000]]
world = pd.DataFrame(data,
columns=['name', 'continent', 'area', 'population', 'gdp']).astype({'name':'object', 'continent':'object', 'area':'Int64', 'population':'Int64', 'gdp':'Int64'})
world

,name,continent,area,population,gdp
0,Afghanistan,Asia,652230,25500100,20343000000
1,Albania,Europe,28748,2831741,12960000000
2,Algeria,Africa,2381741,37100000,188681000000
3,Andorra,Europe,468,78115,3712000000
4,Angola,Africa,1246700,20609294,100990000000


In [ ]:
to_sql(world, 'world')

In [ ]:
sql = '''select t.name, t.population, t.area from world t
where (t.area>= 3000000 or t.population >= 25000000)'''
select(sql)

,name,population,area
0,Afghanistan,25500100,652230
1,Algeria,37100000,2381741


#### 4. Article Views I
Write a solution to find all the authors that viewed at least one of their own articles.
Return the result table sorted by id in ascending order.

In [ ]:
data = [[1, 3, 5, '2019-08-01'],
        [1, 3, 6, '2019-08-02'],
        [2, 7, 7, '2019-08-01'],
        [2, 7, 6, '2019-08-02'],
        [4, 7, 1, '2019-07-22'],
        [3, 4, 4, '2019-07-21'],
        [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data,
columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})
views

,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
1,1,3,6,2019-08-02
2,2,7,7,2019-08-01
3,2,7,6,2019-08-02
4,4,7,1,2019-07-22
5,3,4,4,2019-07-21
6,3,4,4,2019-07-21


In [ ]:
to_sql(views, 'views')

In [ ]:
sql = '''select distinct t.author_id as id from views t
where t.author_id == t.viewer_id
order by id'''
select(sql)

,id
0,4
1,7


####5. Invalid Tweets
Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

In [ ]:
data = [[1, 'Vote for Biden'], [2, 'Let us make America great again!']]
tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})
tweets

,tweet_id,content
0,1,Vote for Biden
1,2,Let us make America great again!


In [ ]:
to_sql(tweets, 'tweets')

In [ ]:
sql = '''select t.tweet_id from tweets t
where length(t.content) > 15'''
select(sql)

,tweet_id
0,2


####6. Restaurant Growth
You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

In [ ]:
data = [[1, 'Jhon', '2019-01-01', 100],
        [2, 'Daniel', '2019-01-02', 110],
        [3, 'Jade', '2019-01-03', 120],
        [4, 'Khaled', '2019-01-04', 130],
        [5, 'Winston', '2019-01-05', 110],
        [6, 'Elvis', '2019-01-06', 140],
        [7, 'Anna', '2019-01-07', 150],
        [8, 'Maria', '2019-01-08', 80],
        [9, 'Jaze', '2019-01-09', 110],
        [1, 'Jhon', '2019-01-10', 130],
        [3, 'Jade', '2019-01-10', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype({'customer_id':'Int64', 'name':'object', 'visited_on':'datetime64[ns]', 'amount':'Int64'})
customer

,customer_id,name,visited_on,amount
0,1,Jhon,2019-01-01,100
1,2,Daniel,2019-01-02,110
2,3,Jade,2019-01-03,120
3,4,Khaled,2019-01-04,130
4,5,Winston,2019-01-05,110
5,6,Elvis,2019-01-06,140
6,7,Anna,2019-01-07,150
7,8,Maria,2019-01-08,80
8,9,Jaze,2019-01-09,110
9,1,Jhon,2019-01-10,130


In [ ]:
to_sql(customer, 'customer')

In [ ]:
sql = '''
SELECT
    visited_on,
    (SELECT SUM(amount) FROM customer WHERE visited_on BETWEEN date(c.visited_on, '-6 days') AND c.visited_on) AS amount,
    ROUND((SELECT SUM(amount) / 7.0 FROM customer WHERE visited_on BETWEEN date(c.visited_on, '-6 days') AND c.visited_on), 2) AS average_amount
FROM customer c
WHERE visited_on >= (SELECT date(MIN(visited_on), '+6 days') FROM customer)
GROUP BY visited_on;

'''
select(sql)

,visited_on,amount,average_amount
0,2019-01-07 00:00:00,860,122.86
1,2019-01-08 00:00:00,840,120.00
2,2019-01-09 00:00:00,840,120.00
3,2019-01-10 00:00:00,1000,142.86


####7. Replace Employee ID With The Unique Identifier
Write a solution to show the unique ID of each user, If a user does not have a unique ID replace just show null.

In [ ]:
data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})
employee_uni

,id,unique_id
0,3,1
1,11,2
2,90,3


In [ ]:
employees

,id,name
0,1,Alice
1,7,Bob
2,11,Meir
3,90,Winston
4,3,Jonathan


In [ ]:
to_sql(employees, 'employees')
to_sql(employee_uni, 'employees_uni')

In [ ]:
sql = '''select coalesce(u.unique_id, null) as unique_id, t.name from employees t
left join employees_uni u on t.id = u.id'''
select(sql)

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


####8. Delete Duplicate Emails

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

In [ ]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})
person

,id,email
0,1,john@example.com
1,2,bob@example.com
2,3,john@example.com


In [ ]:
to_sql(person, 'person')

In [ ]:
cur = con.cursor()
sql = '''
DELETE FROM person
WHERE id NOT IN (
    SELECT MIN(id)
    FROM person
    GROUP BY email)
'''
cur.executescript(sql)

In [ ]:
sql = '''
SELECT * FROM person'''
select(sql)

,id,email
0,1,john@example.com
1,2,bob@example.com
